In [ ]:
#r "nuget:YahooFinanceApi;" 
#r "nuget:QuantLib;" 
//#r "../Algos/bin/Debug/QuantLib.dll"
using YahooFinanceApi;
using QuantLib;


In [ ]:
TSeries data = new();
var history = await Yahoo.GetHistoricalAsync("TQQQ", DateTime.Today.AddDays(-30), DateTime.Now, Period.Daily);
SMA_Series sma = new(data, 5);
WMA_Series wma = new(data, 5);
EMA_Series ema = new(data, 5);
HMA_Series hma = new(data, 5);
DEMA_Series dema = new(data, 5);
TEMA_Series tema = new(data, 5);
ZLEMA_Series zlema = new(data, 5);
JMA_Series jma = new(data, 5);

Console.WriteLine($"date\t\t Value\t SMA\t WMA\t EMA\t HMA\t DEMA\t TEMA \tZLEMA \tJMA");
foreach (var i in history) {
    data.Add((i.DateTime,  (double)i.Close));

    Console.WriteLine($"{data[^1].t:yyyy-MM-dd}\t {(double)data:f2}\t {(double)sma:f2}\t {(double)wma:f2}\t {(double)ema:f2}\t {(double)hma:f2}\t {(double)dema:f2}\t {(double)tema:f2}\t {(double)zlema:f2}\t {(double)jma:f2}");
}

date		 Value	 SMA	 WMA	 EMA	 HMA	 DEMA	 TEMA 	ZLEMA 	JMA
2022-02-23	 45.14	 45.14	 45.14	 45.14	 45.14	 45.14	 45.14	 45.14	 45.14
2022-02-24	 49.56	 47.35	 48.09	 46.61	 47.10	 47.60	 48.25	 48.09	 48.93
2022-02-25	 51.91	 48.87	 50.00	 48.38	 49.36	 50.21	 51.21	 51.62	 51.56
2022-02-28	 52.46	 49.77	 50.98	 49.74	 51.74	 51.87	 52.73	 52.87	 52.39
2022-03-01	 50.04	 49.82	 50.67	 49.84	 51.97	 51.33	 51.47	 51.30	 50.57
2022-03-02	 52.43	 51.28	 51.54	 50.70	 51.71	 52.27	 52.42	 51.67	 51.97
2022-03-03	 50.24	 51.42	 51.19	 50.55	 51.03	 51.49	 51.18	 51.26	 50.66
2022-03-04	 48.09	 50.65	 50.08	 49.73	 49.55	 49.81	 49.03	 48.76	 48.59
2022-03-07	 42.72	 48.70	 47.44	 47.39	 45.72	 45.89	 44.31	 44.24	 43.90
2022-03-08	 42.13	 47.12	 45.25	 45.64	 42.29	 43.47	 41.97	 41.55	 42.34
2022-03-09	 46.64	 45.96	 45.09	 45.97	 43.05	 44.75	 44.38	 44.55	 45.66
2022-03-10	 45.14	 44.94	 44.81	 45.69	 44.94	 44.69	 44.60	 45.75	 45.36
2022-03-11	 42.34	 43.79	 43.94	 44.58	 44.51	 43.16	 4